In [ ]:
import math
import glob
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval, ImageNormalize
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.time import Time
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astroquery.jplhorizons import Horizons
from photutils import centroid_com,centroid_sources
from photutils import CircularAperture, aperture_photometry
from photutils.aperture import CircularAperture, CircularAnnulus
from astropy.wcs import utils
from scipy.interpolate import UnivariateSpline
from astroquery.vizier import Vizier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
''' complete photometry function to apply to any repository of comets '''
def photometry(cmt, objid):
    path = f'/Users/josephmurtagh/Comets/{cmt}/'
    fitses = []
    starlist = []
    
    ''' read in fits files from desired path and add them to list to be analysed '''
    for name in sorted(glob.glob(path+'*.fits')):
        fitses.append(name)
  
    rows = len(fitses)
    colms = 17
    tbl = Table(np.empty(rows*colms).reshape(rows,colms), names=('image','mjd','rh','delta','alpha','appix','x','y','snr','mag','mag_err','afp','afp_err','afp_zero','afp_zero_err','filter','star'))
    tbl['image'] = tbl['image'].astype(str)
    tbl['filter'] = tbl['filter'].astype(str)
    tbl['star'] = tbl['star'].astype(bool)
    tbl['star'][:] = False

    for i in range(len(fitses)):
        ''' read in the fits file and open up image data and header '''
        fname = fitses[i][fitses[i].find('ztf_'):]
        hdu = fits.open(path+fname)
        imgdiff = hdu[0].data
        hdr = hdu[0].header

        ''' take the time of observation from the fits header and use it to create 
            a date range for the epochs for astroquery by adding on one day.  '''
        dtime = Time(f'{hdr["OBSMJD"]}', format = 'mjd')
        dtime.format = 'iso'
        dtime2 = dtime + timedelta(days=1)

        ''' query jpl horizons using the date range and with the object id found on
            the online jpl horizons service first '''
        obj = Horizons(id=objid, location='I41', epochs={'start':dtime.iso, 'stop':dtime2.iso, 'step':'1d'})
        eph = obj.ephemerides(quantities='1,19,20,24,39') 
        
        #1 = RA + DEC (/decimal degrees, icrf), 19(r) = heliocentric comet dist (/AU), 20(delta) = observer-comet dist (/AU), 24(alpha) = phase angle (/degrees)
        
        ''' using small angle & s=r*theta, calculate the aperture radius in pixels
            for a 10^4km radius aperture. header contains scale of arcsecs/pixel '''
        delta_au = eph['delta'][0] * u.au
        delta_km = delta_au.to(u.km)
        theta_rad = ((10000*u.km)/(delta_km)) * u.rad
        theta_arcs = theta_rad.to(u.arcsec)
        pix = float(theta_arcs / hdr['PIXSCALE'] * (1/u.arcsec))
        
        ''' read in relevant details for calculations from header '''
        exptime = hdr['EXPTIME']
        gain = hdr['GAIN']
        rdnoise = hdr['READNOI']
        dark = hdr['DARKCUR']
        zp = hdr['MAGZP']
        wcs = WCS(hdr)
        
        ''' use RA & DEC guesses from jpl horizons to create skycoord object, turn those 
            into pixel values, and then use centroid_sources to hone in on object in image '''
        coords = SkyCoord(ra=eph['RA'][0], dec=eph['DEC'][0], frame='icrs', unit='deg')
        x, y = wcs.world_to_pixel(coords)
        xcent, ycent = centroid_sources(imgdiff, x, y, box_size=10, centroid_func=centroid_com)
        
        ''' using pixel scale calculated, create circular aperture and annulus around object
            with desired radii '''
        ap_radii=pix
        aperture = CircularAperture((xcent[0],ycent[0]), r=ap_radii)
        annulus_aperture = CircularAnnulus((xcent[0],ycent[0]), r_in=4*pix, r_out=5*pix)
        annulus_mask = annulus_aperture.to_mask(method='center')
        
        ''' perform photometry on image '''
        phot = aperture_photometry(imgdiff, aperture)

        ''' extract 1d data from the 2d image (only care about number of counts and not positions)
            and use them to find 3sigma median of background counts (median and not mean to account 
            for potential bright objects eg galaxies etc skewing the counts) '''
        annulus_data = annulus_mask.multiply(imgdiff)
        annulus_data_1d = annulus_data[annulus_mask.data > 0]
        _, bkg_median, _ = sigma_clipped_stats(annulus_data_1d)
        phot['sky_bkg'] = bkg_median
        
        ''' query vizier (panstarrs catalogue) for potential stars '''
        viztbl = Vizier.query_region(coords, radius=4*theta_arcs, catalog='II/349/ps1')
        objlist = []
        maglist = []
        if viztbl:
            for j in range(len(viztbl[0]['rmag'])):
                if viztbl[0]['rmag'][j] < 20.0:
                    crds = SkyCoord(ra=viztbl[0]['RAJ2000'][j], dec=viztbl[0]['DEJ2000'][j], frame='icrs', unit='deg')
                    xi,yi = wcs.world_to_pixel(crds)
                    objlist.append((float(xi),float(yi)))
                    maglist.append(viztbl[0]['rmag'][j])
        
        ''' calculate sky count and subtract from aperture count '''
        phot['aper_sky'] = bkg_median * aperture.area
        phot['counts_minus_sky'] = phot['aperture_sum'] - phot['aper_sky']
        
        ''' find the uncertainty in the subtracted aperture count by using the ccd eqn denominator, 
            uncert = sqrt(n_target + n_sky + n_dark + rdnoise). convert this from units of per electrons
            to per adu's via teh gain '''
        n_target=phot['counts_minus_sky']
        n_sky=phot['aper_sky']
        n_dark=dark*exptime
        err_electrons=np.sqrt(gain*n_target + gain*n_sky + n_dark*aperture.area + rdnoise*rdnoise*aperture.area)
        err_adus=err_electrons/gain  #gain units electrons/adu
        phot['uncertainty'] = err_adus

        ''' calculate a magnitude from the counts calculated. if the magnitude returns a nan
            value, ignore it and skip to the next image as it will be unusable data. otherwise,
            plot the image centered on the comet with the aperture overplotted and magnitude
            and filter labelled, and append any useful details to lists at end'''
        m = -2.5 * np.log10(phot['counts_minus_sky']) + zp
        tbl['image'][i] = fname
        tbl['mjd'][i] = hdr['OBSMJD']
        tbl['rh'][i] = eph['r'][0]
        tbl['delta'][i] = eph['delta'][0]
        tbl['alpha'][i] = eph['alpha'][0]
        tbl['appix'][i] = pix
        tbl['filter'][i] = hdr['FILTER'][-1]
        tbl['x'][i] = x
        tbl['y'][i] = y
        tbl['snr'][i] = (n_target/err_electrons).value[0]
        
        ''' certain cases where photometry cannot be performed due to issues with magnitude 
            measurement - if this is the case the magnitude calculated will be nan, in this 
            case the value in the comet table will be replaced with -999 for filtering out '''
        if math.isnan(m[0]) == True:
            tbl['mag'][i] = -999.0
            pass
        else:
            ''' add magnitude values to comet table '''
            tbl['mag'][i] = m
            
            ''' find the distance of any vizier found objects via pythagoras from the centroid
                - if it is less than 2x the aperture radius and brighter than the comet mag then 
                flag it as an issue in the comet table, if distance is inside the aperture then 
                also flag it as an issue. also create apertures for overplotting if desired '''
            objlist = np.array(objlist)
            if objlist.size!=0:
                for z in range(len(objlist)):
                    dist = np.sqrt((objlist[z][0] - xcent[0])**2 + (objlist[z][1] - ycent[0])**2)
                    if dist < 2*ap_radii and maglist[z] < m:
                        tbl['star'][i] = True
                    elif dist <= ap_radii:
                        tbl['star'][i] = True
                    aps = CircularAperture(objlist, r=3.)
            
            ''' plot the given exposure with ra and dec axes and zscaling on the colour. limit
                axes around the centroid to get a zoom in on the comet (uncomment if you want to
                see the images of the comet) '''
            fig = plt.figure()
            ax = plt.subplot(projection = wcs)
            plt.imshow(imgdiff, cmap='gray', norm=ImageNormalize(imgdiff, interval=ZScaleInterval()))
            plt.xlim(int(xcent)-50, int(xcent)+50)
            plt.ylim(int(ycent)-50, int(ycent)+50)

            plt.grid(color='w', which='major')
            plt.xlabel('RA')
            plt.ylabel('DEC')

            ''' overplot the aperture and annulus for photometry in white and red respectively, and
                if there are any found objects by vizier, highlight them with a blue circle'''
            ap_patches = aperture.plot(color='white', lw=2)
            ann_patches = annulus_aperture.plot(color='red', lw=2)
            if aps:
                star_plot = aps.plot(color='blue', lw=1.5) 
            
            ''' give plots titles of the exposure dates and create appropriate legend '''
            plt.title(f'{dtime}')
#             plt.legend([ap_patches],[f'Mag: {m[0]:.4}  Filter: {hdr["FILTER"]}'])
#             plt.legend(f'Mag: {m[0]:.4}  Filter: {hdr["FILTER"]}')
            plt.show()
            

            ''' find the index of the value of alpha in the phase function curve. need to 
                round alpha_list to 4dp as there are floating point issues in finding matches 
                otherwise. then find value of phase function for that phase angle '''
            idx = np.where(eph['alpha'][0] == np.round(alpha_list,4))[0][0]
            phasefunc = spl(alpha_list)[idx]

            ''' calculate the afrho value for the specified filter that the image being analysed
                is in (found in the header value) using the magnitude of the sun in said filter. 
                uncertainties are propagated using standard formulae, split into parts as it is 
                very hard to follow the lines of thought of each error on one single line'''
            if hdr['FILTER'][-1] == 'r':
                afrho = (((10**(-0.4 * (m[0] - Msun_r))) *(4 * eph['r'][0]**2 * delta_km**2 )/(rho * u.km)).to(u.cm)).value

                delr = delta_km.value**2 * (eph['r'][0]*u.au).to(u.km).value**2
                a = (2 * delta_km * eph['r_3sigma'][0]).value
                b = (2 * (eph['r'][0] * u.au).to(u.km) * eph['r_3sigma'][0]).value
                c = (delr * np.sqrt((a/delta_km.value**2)**2 + (b/(eph['r'][0] * u.au).to(u.km).value**2)**2))
                d = 4 * c / rho
                e = 2.5 * phot['uncertainty'][0] / (phot['counts_minus_sky'][0] * np.log(10))
                f = 10**(-0.4*(m[0] - Msun_r)) * np.log(10) * np.sqrt(e**2 + 0.02**2) * 0.4
                x = 10**(-0.4*(m[0]-Msun_r))
                y = 4 * delta_km.value**2 * (eph['r'][0]*u.au).to(u.km).value**2 / rho
                g = afrho * np.sqrt((f/x)**2 + (d/y)**2)
                
                tbl['mag_err'][i] = e
                tbl['afp'][i] = afrho
                tbl['afp_err'][i] = g
                tbl['afp_zero'][i] = afrho/phasefunc
                tbl['afp_zero_err'][i] = g/phasefunc
            else:
                afrho = (((10**(-0.4 * (m[0] - Msun_i))) *(4 * eph['r'][0]**2 * delta_km**2 )/(rho * u.km)).to(u.cm)).value

                delr = delta_km.value**2 * (eph['r'][0]*u.au).to(u.km).value**2
                a = (2 * delta_km * eph['r_3sigma'][0]).value
                b = (2 * (eph['r'][0] * u.au).to(u.km) * eph['r_3sigma'][0]).value
                c = (delr * np.sqrt((a/delta_km.value**2)**2 + (b/(eph['r'][0] * u.au).to(u.km).value**2)**2))
                d = 4 * c / rho
                e = 2.5 * phot['uncertainty'][0] / (phot['counts_minus_sky'][0] * np.log(10))
                f = 10**(-0.4*(m[0] - Msun_r)) * np.log(10) * np.sqrt(e**2 + 0.02**2)
                x = 10**(-0.4*(m[0]-Msun_r))
                y = 4 * delta_km.value**2 * (eph['r'][0]*u.au).to(u.km).value**2 / rho
                g = afrho * np.sqrt((f/x)**2 + (d/y)**2)
                
                tbl['mag_err'][i] = e
                tbl['afp'][i] = afrho
                tbl['afp_err'][i] = g
                tbl['afp_zero'][i] = afrho/phasefunc
                tbl['afp_zero_err'][i] = g/phasefunc
                
    return tbl

In [ ]:
''' read in phase function for correcting to 0 degs. use univariate spline to 
    interpolate between every degree phase angle so as to get accurate phase function
    values later on '''
phase = ascii.read('/Users/josephmurtagh/phase.tbl', delimiter=' ')
spl = UnivariateSpline(phase.columns[0], phase.columns[1])
spl.set_smoothing_factor(0)
alpha_list = np.arange(0, 180, 0.0001)

''' magnitudes of the sun in each filter provided by wilmer (2018), aperture radius 
    rho is in units of km '''
Msun_r = -26.93
Msun_i = -27.05
rho = 10000
plt.rcParams.update({'font.size': 12})
''' create tables for data for each comet '''
cmt143_tbl = photometry('143P','90001051')
# cmt123_tbl = photometry('123P','90001007')
# cmt60_tbl = photometry('60P','90000654')